In [18]:
import pandas as pd
import numpy as np

from sklearn.model_selection import cross_val_score, KFold, train_test_split
from sklearn.model_selection import LeaveOneOut
from sklearn.preprocessing import PolynomialFeatures

from sklearn.ensemble import RandomForestRegressor
from sklearn.linear_model import LinearRegression
from sklearn import tree
from sklearn.svm import SVR
import xgboost as xgb
from sklearn.linear_model import Lasso
from sklearn.neural_network import MLPRegressor

from sklearn.metrics import r2_score
from sklearn.metrics import mean_absolute_percentage_error

In [19]:
df = pd.read_csv('SamsungData.csv')
newdf = df.drop(columns=['NAME', 'CO2E'])
features = newdf.to_numpy()
emissions = df['CO2E'].to_numpy()

In [20]:
def modelScore(X, y, model):
    avgR2 = avgSSE = avgMAPE = 0
    #cross validation using K folds
    kFolds = 5
    for i, (trainIndex, testIndex) in enumerate(KFold(n_splits=kFolds, shuffle=True, random_state=42).split(X)):
        X_train = X[trainIndex]
        X_test = X[testIndex]
        y_train = y[trainIndex]
        y_test = y[testIndex]
    

        if model == 'linear regression':
            fittedModel = LinearRegression().fit(X_train, y_train)
            
        elif model == 'decision tree':
            dtModel = tree.DecisionTreeRegressor()
            fittedModel = dtModel.fit(X_train, y_train)
                
        elif model == 'random forest':
            y_train = y_train.flatten()
            RF = RandomForestRegressor()
            fittedModel = RF.fit(X_train, y_train)
            
        elif model == 'support vector regression':
            y_train = y_train.flatten()
            fittedModel = SVR().fit(X_train, y_train)
    
        elif model == 'polynomial regression':
            polyFeatures = PolynomialFeatures()
            polyX_train = polyFeatures.fit_transform(X_train)
            polyX_test = polyFeatures.transform(X_test)
            fittedModel = LinearRegression().fit(polyX_train, y_train)
            X_test = polyX_test

        elif model == 'xgboost':
            xgbModel = xgb.XGBModel()
            fittedModel = xgbModel.fit(X_train, y_train)
        
        elif model =='lasso':
            lassoModel = Lasso()

            fittedModel = lassoModel.fit(X_train, y_train)
        
        elif model =="neural network":
            nn = MLPRegressor()

            fittedModel = nn.fit(X_train, y_train)

        y_pred = fittedModel.predict(X_test)

        #scores
        avgR2 += r2_score(y_test, y_pred)    
        avgSSE += np.sum((y_test - y_pred) ** 2)
        avgMAPE += mean_absolute_percentage_error(y_test, y_pred)



    avgR2, avgSSE, avgMAPE = avgR2 / kFolds, avgSSE / kFolds, avgMAPE / kFolds
   #print results
    """print(model)
    print("R2 = " + str(avgR2))
    print("SSE = " + str(avgSSE))
    print("MAPE = " + str(avgMAPE))"""
    print(str(avgR2))
    print(str(avgSSE))
    print(str(avgMAPE))


In [21]:
modelScore(features, emissions, 'linear regression')

-0.06619907080824869
6373.320577431783
0.27289562962364766


In [22]:
modelScore(features, emissions, 'decision tree')

0.31734976155544414
2054.3540000000003
0.17545066840205034


In [23]:
modelScore(features, emissions, 'random forest')

0.6132423572617756
879.3390898737213
0.12363904122775234


In [24]:
modelScore(features, emissions, 'support vector regression')

-0.3414545738131983
10805.896900688396
0.2596147543732578


In [25]:
modelScore(features, emissions, 'polynomial regression')

-16.83619441848349
125054.0929656862
1.0690758913702914


In [26]:
modelScore(features, emissions, 'xgboost')

0.6883210730897324
671.9336806831786
0.10155256765109352


In [27]:
modelScore(features, emissions, 'lasso')

0.7072522115508122
763.676894340049
0.10457777656664932


In [28]:
modelScore(features, emissions, 'neural network')

-295.94200368420434
764164.1383215849
3.5323311469731102


C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


In [37]:
modelsList = ['linear regression', 'decision tree', 'random forest', 'support vector regression', 'polynomial regression', 'xgboost', 'lasso', 'neural network']
multipleFeatures = df[['DISPLAY (inch)','STORAGE (gb)','MEMORY (gb)','CPU (cores)','CPU SPEED (GHz)','GPU (cores)','GPU BENCHMARK (points)','WEIGHT (gm)','MAIN CAMERA (mp)','FRONT CAMERA (mp)','NUMBER OF CAMERAS (back)','SUM OF MEGAPIXELS','BATTERY (mAh)']].to_numpy()
wholeFeaturesList = ['DISPLAY (inch)','STORAGE (gb)','MEMORY (gb)','CPU (cores)','CPU SPEED (GHz)','GPU (cores)','GPU BENCHMARK (3DMark)','WEIGHT (gm)','MAIN CAMERA (mp)','FRONT CAMERA (mp)','NUMBER OF CAMERAS (back)','SUM OF MEGAPIXELS','BATTERY (mAh)']

In [39]:
for model in modelsList:
    modelScore(multipleFeatures, emissions, model)

-0.06619907080824869
6373.320577431783
0.27289562962364766
0.11833746645377112
1641.0220000000002
0.15956759993970904
0.6310652666116272
822.8969470322769
0.11998138683722641
-0.3414545738131983
10805.896900688396
0.2596147543732578
-16.83619441848349
125054.0929656862
1.0690758913702914
0.6883210730897324
671.9336806831786
0.10155256765109352
0.7072522115508122
763.676894340049
0.10457777656664932
-53.84006426154995
220374.12185248546
1.3155362693807644


C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.12_qbz5n2kfra8p0\LocalCache\local-packages\Python312\site-packages\sklearn\neural_network\_multilayer_perceptron.py:691: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(
C:\Users\emdua\A